In [1]:
%load_ext autoreload
%autoreload 2

# Pretraining backbones with Minerva Learn from Randomness

This notebook provides a demonstration of how to pretrain feature extraction backbones using the Minerva Learn from Randomness model. 

## 1. Introduction

### Learn from Randomness (LFR)

LFR is a self-supervised learning (SSL) method for learning representations without requiring labeled data. 

In [2]:
MAX_STEPS = 10_000

# Dataloaders/Datamodule parameters
DL_BATCH_SIZE=2 ** 8
DL_NUM_WORKERS=4
DL_TRAIN_SAMPLES_PER_CLASS=128_000
DL_VAL_SAMPLES_PER_CLASS=None

# Data directory
data_dir = "./data"

# Low data regimen evaluation thresholds
TRAIN_SAMPLES_PER_CLASS_LOW_DATA = [
    # 0.01, # ~2.5k samples
    # 0.05, # ~12.5k samples
    # 0.10, # ~25k samples
    # 0.20, # ~50k samples
    # 1.00

    1_250,   # 1% 
    6_400,   # 5%
    12_800,  # 10%
    25_600,  # 20%
    51_200,  # 50%
    128_000  # 100%
] 

### 2.3 Importing basic modules

Let's import the basic modules, such as lightning, torch, minerva, and other utility modules.

In [3]:
import torch
import torchvision
import lightning
import minerva

print(f"PyTorch version: {torch.__version__}")
print(f"torchvision version: {torchvision.__version__}")
print(f"Lightning version: {lightning.__version__}")

lightning.seed_everything(1969)

/home/igor/Desktop/mo810/course-work/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Seed set to 1969


PyTorch version: 2.7.1+cu126
torchvision version: 0.22.1+cu126
Lightning version: 2.5.1.post0


1969

## <a id="sec_3">3. Setting up the Dataset</a>

In [4]:
from dataset_pcam import PCamDataModule
from pcam.backbone import generate_backbone

datamodule = PCamDataModule(data_dir=data_dir, batch_size=DL_BATCH_SIZE, num_workers=DL_NUM_WORKERS, val_samples_per_class=DL_VAL_SAMPLES_PER_CLASS)
class_names = datamodule.full_dataset.classes
backbone_state_dict = torch.load("logs/PCam/Pretext/checkpoints/backbone.ckpt")

In [5]:
from pcam.benchmark import SGDBenchmark
from torchvision.models.densenet import DenseNet201_Weights

backbone = generate_backbone(DenseNet201_Weights.IMAGENET1K_V1)
downstream_benchmark = SGDBenchmark(datamodule=datamodule, backbone=backbone, train_samples=DL_TRAIN_SAMPLES_PER_CLASS, use_test_set=True)
print("SGD accuracy (ImageNet Weights):", downstream_benchmark.get_accuracy())

SGD accuracy (ImageNet Weights): 0.5970458984375


In [6]:
backbone = generate_backbone()
downstream_benchmark = SGDBenchmark(datamodule=datamodule, backbone=backbone, train_samples=DL_TRAIN_SAMPLES_PER_CLASS, use_test_set=True)
print("SGD accuracy (Default Weights):", downstream_benchmark.get_accuracy())

SGD accuracy (Default Weights): 0.500213623046875


In [7]:
backbone.load_state_dict(backbone_state_dict)
print("SGD accuracy (Pretrained Weights):", downstream_benchmark.get_accuracy())

SGD accuracy (Pretrained Weights): 0.695831298828125


In [8]:
from lightning import LightningModule
from torchmetrics.classification import BinaryAccuracy

class PCamModel(LightningModule):
    def __init__(self, backbone: torch.nn.Module, hidden_dim: int, freeze_backbone: bool = False, validate_each_n_epochs: int = 1):
        super().__init__()

        self.accuracy = BinaryAccuracy()
        
        self.backbone = backbone
        self.prediction_head = torch.nn.Sequential(
            torch.nn.Linear(1920, hidden_dim),
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, 1),
            torch.nn.Sigmoid()
        )

        self.validate_each_n_epochs = validate_each_n_epochs

        if freeze_backbone:
            for param in self.backbone.parameters():
                param.requires_grad = False

    def forward(self, X):
        X = self.backbone(X)
        return self.prediction_head(X).squeeze()
    
    def _single_step(
        self, batch: torch.Tensor, step_name: str
    ) -> torch.Tensor:
        x, y = batch
        outputs = self(x)
        loss = torch.nn.functional.binary_cross_entropy(outputs, y.float())
        preds = (outputs > 0.5).float()
        acc = self.accuracy(preds, y)
        if step_name != "train":
            self.log(f"{step_name}_loss", loss, on_epoch=True, prog_bar=True)
            self.log(f"{step_name}_accuracy", acc, on_epoch=True, prog_bar=True)
        return loss

    def training_step(self, batch: torch.Tensor, batch_idx: int):
        return self._single_step(batch, step_name="train")

    def validation_step(self, batch: torch.Tensor, batch_idx: int):
        if self.current_epoch % self.validate_each_n_epochs == 0:
            self.last_validation_loss = self._single_step(batch, step_name="val")
        
        return self.last_validation_loss

    def test_step(self, batch: torch.Tensor, batch_idx: int):
        return self._single_step(batch, step_name="test")
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3, weight_decay=4e-3)

In [9]:
from lightning import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger, CSVLogger

def evaluate_model(model_name: str, samples: int, freeze_backbone: bool):
    torch.cuda.empty_cache()
    backbone = generate_backbone()
    backbone.load_state_dict(backbone_state_dict)
    hidden_dim = 512

    model = PCamModel(backbone, hidden_dim, freeze_backbone=freeze_backbone, validate_each_n_epochs=20 if samples < 5_000 else 10 if samples < 10_000 else 1)

    log_ckpt_dir=f"logs/PCam/Eval/{model_name}/{samples}_spc/{MAX_STEPS}_steps"

    checkpoint_callback = ModelCheckpoint(dirpath=f"{log_ckpt_dir}/checkpoints", save_weights_only=True, mode='min', monitor='val_loss', save_last="link")
    trainer = Trainer(
        max_steps=MAX_STEPS,
        benchmark=True,
        callbacks=[
            checkpoint_callback, 
            LearningRateMonitor('epoch')
        ],
        logger = [
            TensorBoardLogger(save_dir=log_ckpt_dir, name=model_name),
            CSVLogger(save_dir=log_ckpt_dir, name=model_name)
        ]
    )
    
    trainer.fit(
        model,
        train_dataloaders=datamodule.train_dataloader(samples_per_class=samples),
        val_dataloaders=datamodule.val_dataloader()
    )

    print(f"Loading checkpoint {checkpoint_callback.best_model_path}")
    best_model = PCamModel.load_from_checkpoint(checkpoint_callback.best_model_path, backbone=backbone, hidden_dim=hidden_dim, freeze_backbone=freeze_backbone)
    print("Checkpoint loaded")
    trainer.test(best_model, dataloaders=datamodule.test_dataloader())

In [ ]:
for samples in TRAIN_SAMPLES_PER_CLASS_LOW_DATA:
    evaluate_model(f"PCam-FullFineTune", samples=samples, freeze_backbone=False)
    evaluate_model(f"PCam-FreezeBackbone", samples=samples, freeze_backbone=True)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type           | Params | Mode 
-----------------------------------------------------------
0 | accuracy        | BinaryAccuracy | 0      | train
1 | backbone        | DenseNet       | 18.1 M | train
2 | prediction_head | Sequential     | 985 K  | train
-----------------------------------------------------------
19.1 M    Trainable params
0         Non-trainable params
19.1 M    Total params
76.312    Total estimated model params size (MB)
720       Modules in train mode
0         Modules in eval mode


/home/igor/Desktop/mo810/course-work/.venv/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 999: 100%|██████████| 10/10 [00:05<00:00,  1.78it/s, v_num=0_1, val_loss=1.120, val_accuracy=0.728]

`Trainer.fit` stopped: `max_steps=10000` reached.


Epoch 999: 100%|██████████| 10/10 [00:05<00:00,  1.77it/s, v_num=0_1, val_loss=1.120, val_accuracy=0.728]
Loading checkpoint /home/igor/Desktop/mo810/course-work/logs/PCam/Eval/PCam-FullFineTune/1250_spc/10000_steps/checkpoints/epoch=20-step=210.ckpt


...

Epoch 99: 100%|██████████| 100/100 [00:37<00:00,  2.64it/s, v_num=0_1, val_loss=0.487, val_accuracy=0.764]
Loading checkpoint /home/igor/Desktop/mo810/course-work/logs/PCam/Eval/PCam-FreezeBackbone/12800_spc/10000_steps/checkpoints/epoch=77-step=7800.ckpt


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Checkpoint loaded
Testing DataLoader 0: 100%|██████████| 128/128 [00:25<00:00,  5.08it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.75677490234375      │
│         test_loss         │    0.5037719011306763     │
└───────────────────────────┴───────────────────────────┘

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
